# Fit bmimpf6 data

EoS: PR

Parameter T dependence: Order 3 Polynomial

N (total fitting parameters): 8
-need to work on initialization

## Import Functions

In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../../../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

# parmest (binary_param2)
from bip_fitting_functions import polynomial

import scipy.stats as stats

## Load Data

In [2]:
data_full = pd.read_csv('../r32_bmimpf6_full.csv')
data_subset = pd.read_csv('../r32_bmimpf6_subset.csv')
data_extended = pd.read_csv('../r32-bmimpf6-extended.csv')

## Load Configuration + Check DOF + Load Parameters

In [3]:
lhs_results1 = pd.read_csv('../Data/Fits/LHS_Fits/PR_polyTdep_LHS_from227_1000.csv',index_col=0)
lhs_results1 = lhs_results1[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results2 = pd.read_csv('Data/Init_Final/LHS_Output/PR_polyTdep_LHS_final.csv',index_col=0)
lhs_results2 = lhs_results2[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results3 = pd.read_csv('Data/Init_Final/LHS_Output/PR_polyTdep_LHS_fromquad_final.csv',index_col=0)
lhs_results3 = lhs_results3[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results = pd.concat([lhs_results1,lhs_results2,lhs_results3],axis=0).reset_index(drop=True)

In [4]:
lhs_results.dropna(axis=0)

,sc_param1,sc_param2,sc_param3,sc_param4,sc_param5,sc_param6,sc_param7,sc_param8,SSR
441,0.281367,-1.012734,-0.927035,1.105268,0.742797,-0.022654,-0.030725,-0.031912,907.403250
464,0.281367,-1.012734,-0.927035,1.105268,0.742797,-0.022654,-0.060536,-0.023808,7.306967
9655,-0.496796,0.589789,0.292200,-1.622518,-0.059634,1.001379,0.149753,0.031776,10.405223


In [5]:
lhs_best = lhs_results[lhs_results.SSR == lhs_results.SSR.min()]
lhs_best

,sc_param1,sc_param2,sc_param3,sc_param4,sc_param5,sc_param6,sc_param7,sc_param8,SSR
464,0.281367,-1.012734,-0.927035,1.105268,0.742797,-0.022654,-0.060536,-0.023808,7.306967


In [6]:
sc_param1 = lhs_results.at[464, 'sc_param1']
sc_param2 = lhs_results.at[464, 'sc_param2']
sc_param3 = lhs_results.at[464, 'sc_param3']
sc_param4 = lhs_results.at[464, 'sc_param4']
sc_param5 = lhs_results.at[464, 'sc_param5']
sc_param6 = lhs_results.at[464, 'sc_param6']
sc_param7 = lhs_results.at[464, 'sc_param7']
sc_param8 = lhs_results.at[464, 'sc_param8']

In [7]:
from hfc32_bmimpf6_PR_polynomial import configuration 

In [8]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

The initial DOF is 7


## Fit Parameters

In [9]:
parameters, obj_value, a = polynomial(data_subset, configuration, 'R32', 'bmimpf6', "x_R32", "x_bmimpf6", 
        init_temp =  283.1, init_press = 399300, init_x_c1 = 0.448, init_x_c2 = 0.552,
        init_kappa_1_2A = sc_param2, init_kappa_2_1A = sc_param1,
        init_kappa_1_2B = sc_param4, init_kappa_2_1B = sc_param3,
        init_kappa_1_2C = sc_param6, init_kappa_2_1C = sc_param5,
        init_kappa_1_2D = sc_param8, init_kappa_2_1D = sc_param7,
        eps = 0.1, scaling_fac = 1e-7, 
        filename='Data/Fits/Ipopt_Output/PR_polyTdep_final.txt')

2022-09-19 12:13:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:13:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:13:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:13:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:13:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:13:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:13:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:13:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:13:19 [INFO] idaes

In [10]:
print('Objective (SSE):',obj_value)

Objective (SSE): 7.306967412780702


In [11]:
print('Parameters:',parameters)

Parameters: fs.properties.PR_kappa_A[bmimpf6,R32]    2.265041
fs.properties.PR_kappa_A[R32,bmimpf6]    0.000067
fs.properties.PR_kappa_B[bmimpf6,R32]   -2.356038
fs.properties.PR_kappa_B[R32,bmimpf6]    0.678210
fs.properties.PR_kappa_C[bmimpf6,R32]   -0.557242
fs.properties.PR_kappa_C[R32,bmimpf6]   -1.294428
fs.properties.PR_kappa_D[bmimpf6,R32]    0.555259
fs.properties.PR_kappa_D[R32,bmimpf6]    0.585164
dtype: float64


In [12]:
np.savetxt('Data/Fits/PR_params_polyTdep.csv',parameters,delimiter=',')

In [13]:
cov = np.array(a)
print('Covariance Matrix:')
print(cov)

Covariance Matrix:
[[ 1.04604634e+01  1.07520636e-01 -2.02710538e+01  9.18422203e-02
   9.50156285e+00 -4.80664758e-01  2.79615106e-01  2.80249079e-01]
 [ 1.07520636e-01  1.12958886e-03 -2.10007043e-01  1.52865764e-04
   9.99565534e-02 -3.54616308e-03  2.24743733e-03  2.25404093e-03]
 [-2.02710538e+01 -2.10007043e-01  3.94342283e+01 -1.24167310e-01
  -1.86090962e+01  8.37139493e-01 -4.99945075e-01 -5.01119967e-01]
 [ 9.18422204e-02  1.52865763e-04 -1.24167310e-01  2.64647235e-02
   8.62907235e-03 -4.94426934e-02  2.27732472e-02  2.27767971e-02]
 [ 9.50156285e+00  9.99565534e-02 -1.86090962e+01  8.62907240e-03
   8.88904438e+00 -3.05175739e-01  1.95366826e-01  1.95881695e-01]
 [-4.80664758e-01 -3.54616308e-03  8.37139493e-01 -4.94426934e-02
  -3.05175739e-01  1.01876035e-01 -4.87322939e-02 -4.87614930e-02]
 [ 2.79615106e-01  2.24743733e-03 -4.99945075e-01  2.27732472e-02
   1.95366826e-01 -4.87322939e-02  2.36291946e-02  2.36468313e-02]
 [ 2.80249079e-01  2.25404093e-03 -5.01119967e-01 

In [14]:
np.savetxt('Data/Covariance/PR_Cov_polyTdep.csv',cov,delimiter=',')

## Apply Parameters + Plot

In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(323.15)
m.fs.F101.inlet.pressure.fix(399800)
m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.5)

m.fs.properties.PR_kappa_A['R32','bmimpf6'] = parameters['fs.properties.PR_kappa_A[R32,bmimpf6]']
m.fs.properties.PR_kappa_A['bmimpf6','R32'] = parameters['fs.properties.PR_kappa_A[bmimpf6,R32]']
m.fs.properties.PR_kappa_B['R32','bmimpf6'] = parameters['fs.properties.PR_kappa_B[R32,bmimpf6]']
m.fs.properties.PR_kappa_B['bmimpf6','R32'] = parameters['fs.properties.PR_kappa_B[bmimpf6,R32]']
m.fs.properties.PR_kappa_C['R32','bmimpf6'] = parameters['fs.properties.PR_kappa_C[R32,bmimpf6]']
m.fs.properties.PR_kappa_C['bmimpf6','R32'] = parameters['fs.properties.PR_kappa_C[bmimpf6,R32]']
m.fs.properties.PR_kappa_D['R32','bmimpf6'] = parameters['fs.properties.PR_kappa_D[R32,bmimpf6]']
m.fs.properties.PR_kappa_D['bmimpf6','R32'] = parameters['fs.properties.PR_kappa_D[bmimpf6,R32]']

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'bmimpf6'].unfix()

R32_x = np.zeros((len(data_extended)))
P = np.zeros((len(data_extended)))
T = np.zeros((len(data_extended)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "bmimpf6"] == m.fs.liq)

for i in range(len(data_extended)):
    print('i:',i)
    print('Try x_IL:',data_extended["x_bmimpf6"].iloc[i])
    m.fs.liq = data_extended["x_bmimpf6"].iloc[i]
    m.fs.F101.inlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    m.fs.F101.inlet.pressure.fix(float(data_extended["pressure"].iloc[i]))
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(float(data_extended["x_R32"].iloc[i])+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'bmimpf6'].fix(float(1-(data_extended["x_R32"].iloc[i]+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            R32_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R32']) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
            T[i] = data_extended["temperature"].iloc[i]
        else:
            print('Infeasible Solution')
            R32_x[i] = np.nan
            P[i] = np.nan
            T[i] = data_extended["temperature"].iloc[i]
            
    except ValueError:
        R32_x[i] = np.nan
        P[i] = np.nan
        T[i] = data_extended["temperature"].iloc[i]
        print('Solver Error')

In [ ]:
fitted_data = pd.DataFrame(data=[R32_x,P/1e6,T])
fitted_data = fitted_data.T
fitted_data.columns = ['R32_x','P, MPa','T, K']
fitted_data.to_csv('Data/Fits/PR_FittedData_polyTdep.csv')

In [ ]:
plt.plot(fitted_data.where(fitted_data['T, K']==283.2)['R32_x'],fitted_data.where(fitted_data['T, K']==283.2)['P, MPa'],"b-",label='T=283 K')
plt.plot(fitted_data.where(fitted_data['T, K']==298.1)['R32_x'],fitted_data.where(fitted_data['T, K']==298.1)['P, MPa'],"r-",label='T=298 K')
plt.plot(fitted_data.where(fitted_data['T, K']==323.1)['R32_x'],fitted_data.where(fitted_data['T, K']==323.1)['P, MPa'],"c-",label='T=323 K')
plt.plot(fitted_data.where(fitted_data['T, K']==348.1)['R32_x'],fitted_data.where(fitted_data['T, K']==348.1)['P, MPa'],"m-",label='T=348 K')
plt.plot(data_full["x_R32"],data_full["pressure"]/1e6,"*",color='orange',label="data")  
plt.title('Isotherm R32-[bmim][pf6]')
plt.ylabel('Pressure (MPa)')
plt.xlabel('x R32')
plt.grid(True)
plt.ylim(0,1.110000)
plt.legend()
plt.show()

### Print Eigvals, Eigvectors

An identifiability analysis can be performed with these as well as a FIM because the covariance matrix is the inverse of the FIM. That means that if there are any eigvals > 0, their eigvecs indicate directions of un-identifiability.

In [ ]:
eigvals, eigvecs = np.linalg.eig(cov)

In [ ]:
eigvals

In [ ]:
eigvecs

In [ ]:
np.savetxt('Data/EigenAnalysis/PR_vals_polyTdep.csv',eigvals,delimiter=',')
np.savetxt('Data/EigenAnalysis/PR_vecs_polyTdep.csv',eigvecs,delimiter=',')

In [ ]:
FIM = np.linalg.inv(cov) 

#A_opt
FIM_trace = np.trace(FIM)
cov_trace = np.trace(cov)

#D_opt
FIM_det = np.linalg.det(FIM)

#E_opt
eigvals_FIM, eigvecs_FIM = np.linalg.eig(FIM)
eigvals_FIM_min = eigvals_FIM.min()
eigvals_FIM_max = eigvals_FIM.max()

np.savetxt('Data/EigenAnalysis/PR_FIMtrdet_polyTdep.csv',[cov_trace,FIM_trace,FIM_det,eigvals_FIM_min,eigvals_FIM_max])
np.savetxt('Data/EigenAnalysis/PR_FIMvals_polyTdep.csv',eigvals_FIM,delimiter=',')
np.savetxt('Data/EigenAnalysis/PR_FIMvecs_polyTdep.csv',eigvecs_FIM,delimiter=',')

## AIC Value

See: https://doi.org/10.1080/21642583.2018.1496042

AIC(k) = Nln(MSE(k))+2k

N: number of data points

MSE(k): mean squared error of the model with k parameters

k: number of parameters

MSE(k) = SSE(k)/N

SSE: sum of squared error of the model with k parameters

In [ ]:
#number of datapoints
N = len(data_subset)
#MSE(k)
MSE = obj_value*1E-2/N
#number of parameters
k = 8

In [ ]:
AIC = N*np.log(MSE) + 2*k
AIC

In [ ]:
AIC_data = pd.DataFrame(data=[N,MSE,k,obj_value,AIC])
AIC_data = AIC_data.T
AIC_data.columns = ['N','MSE','k','Objective, Pa^2','AIC']
AIC_data.to_csv('Data/AIC/PR_AIC_polyTdep.csv')